# 🧪 Padronização de SMILES

## 🔍 O que é SMILES?

**SMILES** (Simplified Molecular Input Line Entry System) é uma **representação textual de moléculas químicas**.  
Em vez de usar uma estrutura gráfica, as moléculas são codificadas como uma cadeia de caracteres. Por exemplo:

- Etanol: `CCO`  
- Ácido acético: `CC(=O)O`

Essa representação é amplamente usada em:
- Química computacional
- Bancos de dados moleculares
- Modelos de machine learning aplicados à descoberta de fármacos

---

## 🎯 Por que padronizar SMILES?

A **mesma molécula pode ter diferentes representações SMILES**.  
Por exemplo, o etanol pode ser representado como `CCO` ou `OCC`.

A padronização (ou "normalização") garante que:

- ✅ Moléculas idênticas sejam tratadas como iguais  
- 🧹 Evite duplicatas em bancos de dados  
- 🔍 Facilite buscas, comparações e análises estatísticas  
- 🤖 Melhore o desempenho de modelos de machine learning  

---

## 🛠️ O que a padronização pode incluir?

- Remoção de cargas desnecessárias  
- Ajuste de hidrôgenos explícitos  
- Geração de SMILES canônicos (ordem padrão dos átomos)  
- Conversão para uma forma tautomérica comum  
- Neutralização de moléculas onde aplicável  

---




# 📦 Instalação e Importação de Módulos RDKit

Antes de começar a trabalhar com moléculas, precisamos instalar e importar as bibliotecas necessárias.

# 🔬 O que é o RDKit?

**RDKit** é uma biblioteca open-source amplamente utilizada em **química computacional**, **quimioinformática** e **descoberta de fármacos**.

Ela fornece ferramentas poderosas para:

- 🧪 Manipular estruturas químicas
- 🧠 Extrair propriedades moleculares (peso molecular, logP, número de anéis, etc.)
- 🧼 Padronizar e limpar moléculas
- 🧬 Gerar e comparar **fingerprints moleculares**
- 🎯 Calcular similaridade entre compostos
- 🧰 Realizar reações químicas simuladas
- 🧱 Gerar e manipular representações SMILES e InChI
- 🌐 Integrar com machine learning (ex: usando vetores químicos como input de modelos)



In [40]:
# --------------------
# 📦 Instalar o RDKit
# --------------------
!pip install rdkit

In [41]:
# ---------------------------------------------------
# 📚 Importar os módulos necessários
# ---------------------------------------------------

# Importa o módulo principal do RDKit para manipular moléculas
from rdkit import Chem

# Desativa os avisos e mensagens de log do RDKit (para deixar o output mais limpo)
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# Módulos adicionais do RDKit:
from rdkit.Chem import (
    Descriptors,   # Cálculo de propriedades moleculares (peso molecular, logP, etc.)
    Draw,          # Visualização de estruturas químicas
    AllChem,       # Ferramentas para química 3D, fingerprints e conformações
    DataStructs,   # Manipulação de fingerprints e similaridade
    inchi          # Conversão para InChI e InChIKey
)

# Importa a ferramenta de padronização molecular do RDKit
from rdkit.Chem.MolStandardize import rdMolStandardize

# Utilitário do Python: contador de elementos (muito útil em análises de datasets químicos)
from collections import Counter

# Utilitário do Jupyter/Colab para exibir resultados de forma interativa (ex: estruturas químicas)
from IPython.display import display

In [42]:
# ---------------------------------------------------
# ✅ Função para validar critérios básicos de SMILES
# ---------------------------------------------------

def check_criteria(smiles):
    """
    Verifica se uma string SMILES atende aos critérios mínimos
    para ser processada na pipeline (pré-processamento químico).

    Critérios verificados:
    - SMILES válido (conversível para molécula do RDKit)
    - Contém pelo menos 1 átomo de carbono
    - Não é uma mistura de compostos
    - Contém apenas elementos permitidos (C, H, N, O, P, S, F, Cl, Br, I)
    - Peso molecular menor ou igual a 1500 Da

    Retorna:
        bool: True se a molécula atender a todos os critérios, False caso contrário
    """

    try:
        # 🧪 Converte a string SMILES em um objeto de molécula do RDKit
        mol = Chem.MolFromSmiles(smiles)

        # Se a conversão falhar (SMILES inválido), rejeita
        if mol is None:
            print(f"SMILES inválido: {smiles}")
            return False

        # 🔍 Verifica se há pelo menos um átomo de carbono
        if not mol.HasSubstructMatch(Chem.MolFromSmarts('C')):
            print(f"Rejeitado (sem carbono): {smiles}")
            return False

        # ⚠️ Verifica se é uma mistura (função externa necessária: is_mixture)
        if is_mixture(mol):
            print(f"Rejeitado (mistura detectada): {smiles}")
            return False

        # 🔬 Lista de elementos químicos permitidos
        allowed_elements = ['C', 'H', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I']

        # Verifica se todos os átomos pertencem à lista permitida
        if any(atom.GetSymbol() not in allowed_elements for atom in mol.GetAtoms()):
            print(f"Rejeitado (elementos não permitidos ou metais): {smiles}")
            return False

        # ⚖️ Verifica se o peso molecular está dentro do limite
        mol_weight = Descriptors.MolWt(mol)
        if mol_weight > 1500:
            print(f"Rejeitado (Peso molecular {mol_weight:.2f} > 1500 Da): {smiles}")
            return False

        # ✅ Se passou por todos os filtros
        print(f"SMILES aceito: {smiles}")
        return True

    except Exception as e:
        # Captura erros inesperados durante o processamento
        print(f"Erro ao processar SMILES {smiles}: {e}")
        return False


In [43]:
# ---------------------------------------------------
# 🔍 Função para identificar possíveis misturas químicas
# ---------------------------------------------------

def is_mixture(mol):
    """
    Verifica se a molécula contém mais de 5 fragmentos com carbono,
    o que pode indicar que se trata de uma mistura (e não de um composto único).

    Parâmetros:
        mol (rdkit.Chem.Mol): Objeto de molécula do RDKit

    Retorna:
        bool: True se a molécula for provavelmente uma mistura, False caso contrário
    """

    # 🧩 Fragmenta a molécula em partes desconectadas (sem sanitizar)
    # Exemplo: sais, aditivos ou moléculas ligadas por íons podem se separar aqui
    fragments = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=False)

    # 🔎 Filtra apenas os fragmentos que contêm ao menos um átomo de carbono
    carbon_fragments = [
        frag for frag in fragments
        if frag.HasSubstructMatch(Chem.MolFromSmarts('C'))
    ]

    # ✅ Retorna True se houver mais de 5 fragmentos com carbono (possível mistura)
    return len(carbon_fragments) > 5


In [44]:
# ---------------------------------------------------
# ⚗️ Função para neutralizar moléculas (remover cargas)
# ---------------------------------------------------

def neutralize_smiles(smiles):
    """
    Neutraliza cargas em moléculas (ácidos, bases, sais, etc.)
    usando o 'Uncharger' do RDKit.

    Parâmetros:
        smiles (str): SMILES da molécula a ser neutralizada

    Retorna:
        str: SMILES neutralizado, ou o original se a conversão falhar
    """
    try:
        # 🛠️ Exibe o SMILES que está sendo processado (útil para debug)
        print(f"Attempting to neutralize: {smiles}")

        # 🔄 Converte a string SMILES para objeto molécula RDKit
        mol = Chem.MolFromSmiles(smiles)

        # ❌ Se a conversão falhar (mol = None), retorna o SMILES original
        if mol is None:
            return smiles

        # 🧼 Cria um objeto "Uncharger" para remover cargas
        uncharger = rdMolStandardize.Uncharger()

        # ⚡ Aplica a neutralização na molécula
        mol = uncharger.uncharge(mol)

        # 🔁 Converte de volta para string SMILES (forma neutralizada)
        return Chem.MolToSmiles(mol)

    except Exception as e:
        # ⚠️ Caso ocorra algum erro, exibe o erro e retorna o SMILES original
        print(f"Error neutralizing {smiles}: {e}")
        return smiles


In [45]:
# ---------------------------------------------------
# 🧬 Função para converter SMILES em InChIKey
# ---------------------------------------------------

def smiles_to_inchikey(smiles):
    """
    Converte uma string SMILES para o seu InChIKey correspondente,
    que é um identificador único para substâncias químicas.

    Parâmetros:
        smiles (str): Representação SMILES da molécula

    Retorna:
        str: InChIKey se a conversão for bem-sucedida; "Cannot_do" se falhar
    """
    try:
        # 🔄 Converte o SMILES em um objeto de molécula do RDKit
        mol = Chem.MolFromSmiles(smiles)

        # 🔬 Gera a string InChI (formato padronizado pelo IUPAC)
        inchi_string = Chem.MolToInchi(mol)

        # 🔑 Converte a InChI em um InChIKey (formato curto, fixo e único)
        inchikey = Chem.inchi.InchiToInchiKey(inchi_string)

        # ✅ Exibe o resultado no console
        print(f"Converted SMILES to InChIKey: {inchikey}")

        return inchikey

    except Exception as e:
        # ⚠️ Se der erro na conversão, mostra o erro e retorna "Cannot_do"
        print(f"Error converting SMILES {smiles}: {e}")
        return "Cannot_do"


In [46]:
# ---------------------------------------------------
# 🧼 Função de padronização completa de moléculas
# ---------------------------------------------------

def standardize(smiles):
    """
    Realiza a limpeza e padronização completa de uma molécula usando RDKit.

    O processo inclui:
    - Conversão bidirecional InChI <-> Mol
    - Limpeza básica
    - Remoção de fragmentos pequenos
    - Neutralização de cargas
    - Remoção de informações desnecessárias (cargas, isótopos, estereoquímica)
    - Canonicalização de tautômeros
    - Iterações até estabilizar o SMILES

    Parâmetros:
        smiles (str): String SMILES de entrada

    Retorna:
        str: SMILES padronizado final ou "Cannot_do" em caso de erro
    """
    try:
        print(f"\n---- Standardizing: {smiles} ----")

        # 🔄 Converte SMILES para molécula RDKit
        mol = Chem.MolFromSmiles(smiles)

        # 👁️ Mostra a estrutura original
        display(Draw.MolToImage(mol))

        # Contador para controlar quantas vezes cada SMILES aparece após limpeza
        smiles_clean_counter = Counter()

        # Dicionário para guardar moléculas únicas
        mol_dict = {}

        # Flag para saber se a molécula foi estabilizada
        is_finalize = False

        # 🔁 Repetir processo até 5 vezes (caso mudanças ainda ocorram)
        for i in range(5):
            print(f"\nIteration {i+1}:")

            # 🔄 Conversão SMILES → InChI → Mol (para padronizar estrutura interna)
            inchi_standardized = Chem.MolToInchi(mol)
            mol = Chem.MolFromInchi(inchi_standardized)
            display(Draw.MolToImage(mol))

            # 🧼 Etapa 1: limpeza geral da molécula
            mol = rdMolStandardize.Cleanup(mol)
            display(Draw.MolToImage(mol))

            # ✂️ Etapa 2: remover fragmentos pequenos (deixa apenas o maior)
            mol = rdMolStandardize.FragmentParent(mol)
            display(Draw.MolToImage(mol))

            # ⚡ Etapa 3: neutralizar cargas (ácidos/bases)
            mol = rdMolStandardize.Uncharger().uncharge(mol)
            display(Draw.MolToImage(mol))

            # 🧽 Etapa 4: remover informações extras
            mol = rdMolStandardize.ChargeParent(mol)   # Remove alterações artificiais de carga
            mol = rdMolStandardize.IsotopeParent(mol)  # Remove isótopos
            mol = rdMolStandardize.StereoParent(mol)   # Remove estereoquímica
            display(Draw.MolToImage(mol))

            # 🔁 Etapa 5: canonicalização de tautômeros
            mol = rdMolStandardize.TautomerEnumerator().Canonicalize(mol)
            display(Draw.MolToImage(mol))

            # 🔁 Converte de volta para SMILES padronizado
            smiles_standardized = Chem.MolToSmiles(mol)
            print(f"Standardized SMILES: {smiles_standardized}")

            # ✅ Verifica se o SMILES se estabilizou (parou de mudar)
            if smiles == smiles_standardized:
                print("No further changes detected. Standardization complete.")
                is_finalize = True
                break

            # 📊 Atualiza contagem de ocorrências do SMILES gerado
            smiles_clean_counter[smiles_standardized] += 1
            if smiles_standardized not in mol_dict:
                mol_dict[smiles_standardized] = mol

            # 🔁 Atualiza molécula e SMILES para próxima iteração
            smiles = smiles_standardized
            mol = Chem.MolFromSmiles(smiles)

        # 🧠 Se após 5 iterações ainda houver instabilidade, escolhe o SMILES mais comum
        if not is_finalize:
            smiles_standardized = smiles_clean_counter.most_common(1)[0][0]
            print("Most common SMILES selected as final.")

        # ✅ Exibe estrutura final
        print(f"\nFinal SMILES: {smiles_standardized}")
        display(Draw.MolToImage(Chem.MolFromSmiles(smiles_standardized)))

        return smiles_standardized

    except Exception as e:
        print(f"Error standardizing {smiles}: {e}")
        return "Cannot_do"



In [47]:
# Importar bibliotecas necessárias
import pandas as pd
from rdkit import Chem

# -----------------------------------------
# 📂 Carregar arquivo CSV com SMILES
# -----------------------------------------
df = pd.read_csv("/content/Voriconazol.csv")  # Substitua pelo caminho correto do seu arquivo

# -----------------------------------------
# ✅ Validar SMILES e salvar status
# -----------------------------------------
# Aplica uma função que verifica se o SMILES é válido usando RDKit
# Se a conversão para Mol for bem-sucedida, marca como "Valid", senão "Invalid"
df['Status'] = df['alignment'].apply(
    lambda s: "Valid" if Chem.MolFromSmiles(s) else "Invalid"
)

# -----------------------------------------
# 📊 Contagem de SMILES válidos e inválidos
# -----------------------------------------
valid_count = df['Status'].value_counts().get('Valid', 0)
invalid_count = df['Status'].value_counts().get('Invalid', 0)

print(f"Valid SMILES: {valid_count}")
print(f"Invalid SMILES: {invalid_count}")



Valid SMILES: 2339
Invalid SMILES: 0


In [48]:
# ---------------------------------------------------
# Função para canonicalizar uma string SMILES
# ---------------------------------------------------

def canonicalize_smiles(smiles):
    try:
        # Converte o SMILES em objeto molécula RDKit
        mol = Chem.MolFromSmiles(smiles)

        # Se a conversão foi bem-sucedida, retorna o SMILES canônico
        # Caso contrário, retorna None
        return Chem.MolToSmiles(mol, canonical=True) if mol else None

    except:
        # Caso ocorra algum erro na conversão, retorna None
        return None

# ---------------------------------------------------
# Aplicar canonicalização para todos os SMILES na coluna 'alignment'
# ---------------------------------------------------

df['SMILES_Canonical'] = df['alignment'].apply(canonicalize_smiles)



In [49]:
# ---------------------------------------------------
# Filtrar linhas onde o SMILES original é diferente do canônico
# ---------------------------------------------------
df_diff = df[df['alignment'] != df['SMILES_Canonical']].copy()

# Contar quantos SMILES foram modificados e total de SMILES no dataset
num_changed = len(df_diff)
total = len(df)

print(f"{num_changed} de {total} SMILES foram modificados após a canonicalização.")

# Se houver SMILES modificados, mostrar alguns exemplos e salvar em CSV
if num_changed > 0:
    print("\nExemplos de SMILES modificados (original vs canônico):")
    print(df_diff[['alignment', 'SMILES_Canonical']].head(10))  # Mostra os primeiros 10 exemplos

    # Salvar arquivo CSV com os SMILES modificados para análise posterior
    df_diff.to_csv("smiles_modified.csv", index=False)
    print("Arquivo 'smiles_modified.csv' salvo.")



2339 de 2339 SMILES foram modificados após a canonicalização.

Exemplos de SMILES modificados (original vs canônico):
                                           alignment  \
0   C[C@@H](c1ncncc1F)[C@@](O)(Cn2cncn2)c3ccc(cc3F)F   
1    C[C@@H](c1ncncc1F)[C@](O)(Cn2cncn2)c3ccc(cc3F)F   
2    C[C@H](c1ncncc1F)[C@@](O)(Cn2cncn2)c3ccc(cc3F)F   
3     C[C@H](c1ncncc1F)[C@](O)(Cn2cncn2)c3ccc(cc3F)F   
4     C[C@H](c1ncncc1F)[C@@](O)(Cn2cncn2)c3ccc(cc3)F   
5   C[C@@H](c1ncncc1F)[C@@](F)(Cn2cncn2)c3ccc(cc3F)F   
6    C[C@@H](c1ncncc1F)[C@](F)(Cn2cncn2)c3ccc(cc3F)F   
7    C[C@H](c1ncncc1F)[C@@](F)(Cn2cncn2)c3ccc(cc3F)F   
8     C[C@H](c1ncncc1F)[C@](F)(Cn2cncn2)c3ccc(cc3F)F   
9  C[C@@H](c1ncnc(c1F)Cl)[C@@](O)(Cn2cncn2)c3ccc(...   

                                    SMILES_Canonical  
0   C[C@@H](c1ncncc1F)[C@@](O)(Cn1cncn1)c1ccc(F)cc1F  
1    C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F  
2    C[C@H](c1ncncc1F)[C@@](O)(Cn1cncn1)c1ccc(F)cc1F  
3     C[C@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc

In [50]:
# ---------------------------------------------------
# Remover duplicatas com base na coluna 'SMILES_Canonical'
# Mantém a primeira ocorrência de cada SMILES canônico
# ---------------------------------------------------
df_no_duplicates = df.drop_duplicates(subset=['SMILES_Canonical'])

# Calcular quantos duplicados foram removidos
num_duplicates = len(df) - len(df_no_duplicates)

print(f"Número de duplicatas removidas: {num_duplicates}")

# ---------------------------------------------------
# Salvar os datasets em arquivos CSV
# ---------------------------------------------------

# Salva o dataset original (com SMILES canônicos e status)
df.to_csv("validation_smiles_canonical.csv", index=False)
print("Arquivo 'validation_smiles_canonical.csv' salvo.")

# Salva o dataset sem duplicatas
df_no_duplicates.to_csv("validation_smiles_no_duplicates.csv", index=False)
print("Arquivo 'validation_smiles_no_duplicates.csv' salvo sem duplicatas.")



Número de duplicatas removidas: 0
Arquivo 'validation_smiles_canonical.csv' salvo.
Arquivo 'validation_smiles_no_duplicates.csv' salvo sem duplicatas.


---

## 📚 Referências e Créditos

Este notebook adota e adapta trechos de código desenvolvidos por especialistas da comunidade de química computacional.

### ✍️ Autores:

- **Srijit Seal**  
  Site pessoal: [srijitseal.com](https://srijitseal.com)  
  Blog: [Standardizing a molecule using RDKit](https://bitsilla.com/blog/2021/06/standardizing-a-molecule-using-rdkit/)

- **Lewis Mervin** (REINVENT Project Contributor)  
  Repositório: [REINVENT4 GitHub](https://github.com/lewismervin1/REINVENT4/tree/main/reinvent/chemistry/standardization)

Essas fontes foram utilizadas para construir os trechos de código de padronização química com a biblioteca **RDKit**, adaptados neste notebook para fins educacionais.

---

⚠️ **Nota**: Este material é apenas para fins acadêmicos e educacionais. O código original pertence aos respectivos autores e está disponível sob suas respectivas licenças.

---